In [2]:
from IPython.display import display, HTML

display(HTML(""" 
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>CupCode – Cafe Billing App</title>
<style>
body {
    font-family: Poppins, sans-serif;
    background: #f7f3ef;
    margin: 0;
    padding: 0;
}
.container {
    width: 90%;
    max-width: 800px;
    margin: 30px auto;
    background: #fff;
    border-radius: 12px;
    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
    padding: 20px 30px;
}
h1 {
    text-align: center;
    color: #6b3e26;
}
h2 {
    color: #503220;
}
table {
    width: 100%;
    border-collapse: collapse;
    margin-bottom: 20px;
}
th, td {
    border-bottom: 1px solid #ddd;
    padding: 10px;
    text-align: center;
}
input[type="number"] {
    width: 60px;
    padding: 5px;
    border: 1px solid #ccc;
    border-radius: 5px;
}
button {
    background-color: #6b3e26;
    color: white;
    border: none;
    padding: 10px 20px;
    border-radius: 8px;
    font-size: 16px;
    cursor: pointer;
    margin: 5px;
}
button:hover {
    background-color: #8b5437;
}
.bill-section {
    background-color: #fffaf4;
    border-radius: 10px;
    padding: 15px;
    text-align: center;
}
.bill-section p {
    font-size: 18px;
    margin: 5px 0;
    color: #333;
}
</style>
</head>
<body>
<div class="container">
    <h1>☕ CupCode – Cafe Billing App</h1>
    <h3 style="text-align:center; color:#555;">Developed by Khushi S Pillay</h3>
    <table id="menuTable">
        <thead>
            <tr><th>Item</th><th>Price (₹)</th><th>Quantity</th><th>Total (₹)</th></tr>
        </thead>
        <tbody>
            <tr><td>Cappuccino</td><td>120</td><td><input type="number" min="0" value="0" class="qty"></td><td class="item-total">0</td></tr>
            <tr><td>Espresso</td><td>100</td><td><input type="number" min="0" value="0" class="qty"></td><td class="item-total">0</td></tr>
            <tr><td>Cold Coffee</td><td>150</td><td><input type="number" min="0" value="0" class="qty"></td><td class="item-total">0</td></tr>
            <tr><td>Chocolate Muffin</td><td>80</td><td><input type="number" min="0" value="0" class="qty"></td><td class="item-total">0</td></tr>
            <tr><td>Cheese Sandwich</td><td>90</td><td><input type="number" min="0" value="0" class="qty"></td><td class="item-total">0</td></tr>
        </tbody>
    </table>
    <button id="calculateBtn">💰 Calculate Bill</button>
    <button id="printBtn">🧾 Print / Download Receipt</button>

    <div class="bill-section">
        <h2>Bill Summary</h2>
        <p id="totalAmount">Total: ₹0</p>
        <p id="gstAmount">GST (5%): ₹0</p>
        <p id="grandTotal">Grand Total: ₹0</p>
    </div>
</div>

<script>
document.getElementById("calculateBtn").addEventListener("click", function() {
    const rows = document.querySelectorAll("#menuTable tbody tr");
    let total = 0;

    rows.forEach(row => {
        const price = parseFloat(row.cells[1].textContent);
        const qty = parseInt(row.querySelector(".qty").value) || 0;
        const itemTotal = price * qty;
        row.querySelector(".item-total").textContent = itemTotal.toFixed(2);
        total += itemTotal;
    });

    const gst = total * 0.05;
    const grandTotal = total + gst;

    document.getElementById("totalAmount").textContent = `Total: ₹${total.toFixed(2)}`;
    document.getElementById("gstAmount").textContent = `GST (5%): ₹${gst.toFixed(2)}`;
    document.getElementById("grandTotal").textContent = `Grand Total: ₹${grandTotal.toFixed(2)}`;

    // Store for printing
    localStorage.setItem("billData", JSON.stringify({
        total, gst, grandTotal, items: getItemsData(rows)
    }));
});

function getItemsData(rows) {
    const items = [];
    rows.forEach(row => {
        const name = row.cells[0].textContent;
        const price = parseFloat(row.cells[1].textContent);
        const qty = parseInt(row.querySelector(".qty").value) || 0;
        const total = parseFloat(row.querySelector(".item-total").textContent);
        if (qty > 0) items.push({ name, price, qty, total });
    });
    return items;
}

// Print Receipt
document.getElementById("printBtn").addEventListener("click", () => {
    const data = JSON.parse(localStorage.getItem("billData") || "{}");
    if (!data.items || data.items.length === 0) {
        alert("Please calculate the bill before printing.");
        return;
    }

    const win = window.open("", "_blank");
    const now = new Date().toLocaleString();

    win.document.write(`
        <html>
        <head>
            <title>CupCode Receipt</title>
            <style>
                body { font-family: Poppins, sans-serif; background: #fffaf4; padding: 20px; }
                h2 { text-align: center; color: #6b3e26; }
                table { width: 100%; border-collapse: collapse; margin-top: 15px; }
                th, td { border: 1px solid #ccc; padding: 8px; text-align: center; }
                .summary { text-align: right; margin-top: 20px; font-size: 16px; }
                .footer { text-align: center; margin-top: 30px; color: #6b3e26; }
            </style>
        </head>
        <body>
            <h2>☕ CupCode – Cafe Billing App</h2>
            <p style="text-align:center;">Developed by <b>Khushi S Pillay</b></p>
            <p>Date & Time: ${now}</p>
            <table>
                <tr><th>Item</th><th>Price</th><th>Qty</th><th>Total</th></tr>
                ${data.items.map(item => 
                    `<tr>
                        <td>${item.name}</td>
                        <td>₹${item.price}</td>
                        <td>${item.qty}</td>
                        <td>₹${item.total.toFixed(2)}</td>
                    </tr>`
                ).join('')}
            </table>
            <div class="summary">
                <p>Total: ₹${data.total.toFixed(2)}</p>
                <p>GST (5%): ₹${data.gst.toFixed(2)}</p>
                <p><strong>Grand Total: ₹${data.grandTotal.toFixed(2)}</strong></p>
            </div>
            <div class="footer">
                <p>Thank you for visiting CupCode Cafe ☕</p>
                <p>Have a great day!</p>
            </div>
            <script>window.print();</script>
        </body>
        </html>
    `);
    win.document.close();
});
</script>
</body>
</html>
"""))
